In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [109]:
from pathlib import Path
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from salt_utils import *
from datasets import SaltDatasetArrays, get_dls
from unet import dpnunet68
from train import loss_batch, fit
from loss import lovasz_hinge

In [80]:
train_augs = get_tfms(128, val=False)
val_augs = get_tfms(128, val=True)

In [81]:
PATH = Path('/Users/egaasedelen/Data/salt/')
train_img_paths, train_mask_paths, val_img_paths, val_mask_paths = next(cross_val_paths(PATH))

In [82]:
#Need Data
trn_dl, val_dl = get_dls(train_img_paths, train_mask_paths, val_img_paths, val_mask_paths)

In [83]:
#We also need an arch
arch = dpnunet68()

In [84]:
x, y = next(iter(trn_dl))

In [85]:
y_ = arch(torch.autograd.Variable(x.permute(0,3,1,2))) #hell yes it works.

In [86]:
#We need a loss function

In [87]:
loss = lovasz_hinge

In [88]:
loss(y_, y)

Variable containing:
 2.1725
[torch.FloatTensor of size 1]

In [89]:
opt = torch.optim.SGD(arch.parameters(), lr=0.0001, momentum=0.9)

In [92]:
l, _, _ = loss_batch(arch, torch.autograd.Variable(x, requires_grad=True), y, loss, opt=opt)

In [110]:
fit(1, arch, loss, opt, trn_dl, val_dl)





  0%|          | 0/3200 [00:00<?, ?it/s]



Epoch 0:   0%|          | 0/3200 [00:00<?, ?it/s]



Epoch 0:   0%|          | 1/3200 [00:00<50:24,  1.06it/s]



Epoch 0:   0%|          | 1/3200 [00:00<50:24,  1.06it/s, loss=2.12893]



Epoch 0:   0%|          | 2/3200 [00:01<45:34,  1.17it/s, loss=2.12893]



Epoch 0:   0%|          | 2/3200 [00:01<45:34,  1.17it/s, loss=2.05154]



Epoch 0:   0%|          | 3/3200 [00:02<41:07,  1.30it/s, loss=2.05154]



Epoch 0:   0%|          | 3/3200 [00:02<41:07,  1.30it/s, loss=1.97088]



Epoch 0:   0%|          | 4/3200 [00:02<38:16,  1.39it/s, loss=1.97088]



Epoch 0:   0%|          | 4/3200 [00:02<38:16,  1.39it/s, loss=2.11416]



Epoch 0:   0%|          | 5/3200 [00:03<36:45,  1.45it/s, loss=2.11416]



Epoch 0:   0%|          | 5/3200 [00:03<36:45,  1.45it/s, loss=2.06402]



Epoch 0:   0%|          | 6/3200 [00:03<35:29,  1.50it/s, loss=2.06402]



Epoch 0:   0%|          | 6/3200 [00:04<35:29,  1.50it/s, loss=2.11059]



Epoch 0:   0%

KeyboardInterrupt
Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x1c5d9aa0f0>>
Traceback (most recent call last):
  File "/Users/egaasedelen/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 331, in __del__
    def __del__(self):
  File "/Users/egaasedelen/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 175, in handler
    _error_if_any_worker_fails()
RuntimeError: DataLoader worker (pid 98970) exited unexpectedly with exit code 1.


KeyboardInterrupt: 